# LSST Twilight Planner (Modular) — Notebook

This notebook demonstrates the use of the `twilight_planner_pkg` package to
generate twilight observing plans. The workflow is:

1. Set up imports and paths for the input SN catalog and output directory.
2. Build a fully specified `PlannerConfig` object with all optional parameters
   shown (defaults are used unless otherwise noted).
3. Run the planner and inspect the CSV outputs written to `OUTDIR`.


In [ ]:
# --- Imports and path setup ---
import sys
import os

# Ensure local package is importable
sys.path.append("./")

from twilight_planner_pkg.config import PlannerConfig
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

# Paths: adjust if your data/output lives elsewhere
CSV_PATH = "../data/ATLAS_2021_to25_cleaned.csv"
OUTDIR = "../twilight_outputs"
os.makedirs(OUTDIR, exist_ok=True)

print("CSV exists:", os.path.exists(CSV_PATH))
print("Output dir:", OUTDIR)

## Configure the planner

The next cell defines a complete `PlannerConfig`. Every parameter of the
dataclass is included so the notebook doubles as documentation. Edit any value
to explore different strategies or hardware assumptions.


In [9]:
# ---- User-editable parameters ----

# Date range (UTC)
START_DATE = "2024-01-01"
END_DATE = "2024-01-03"
RUN_LABEL = None

# -- Site --
LAT_DEG = -30.2446
LON_DEG = -70.7494
HEIGHT_M = 2663

# -- Visibility --
MIN_ALT_DEG = 20.0
TWILIGHT_SUN_ALT_MIN_DEG = -15.0  # minimum altitude for twilight
TWILIGHT_SUN_ALT_MAX_DEG = -5.0  # maximum altitude for twilight


# -- Filters and hardware --
FILTERS = ["g", "r", "i", "z"]
CAROUSEL_CAPACITY = 5
FILTER_CHANGE_S = 120.0
READOUT_S = 2.0
FILTER_CHANGE_TIME_S = None  # legacy alias
READOUT_TIME_S = None  # legacy alias
INTER_EXPOSURE_MIN_S = 15.0
EXPOSURE_BY_FILTER = {"g": 5.0, "r": 5.0, "i": 5.0, "z": 5.0}
# NEW — cadence knobs
CADENCE_ENABLE = True
CADENCE_PER_FILTER = True
CADENCE_DAYS_TARGET = 3.0
CADENCE_JITTER_DAYS = 0.25
CADENCE_DAYS_TOLERANCE = 0.5
CADENCE_BONUS_SIGMA_DAYS = 0.5
CADENCE_BONUS_WEIGHT = 0.25

# Allow colors in one visit (if you want them)
MAX_FILTERS_PER_VISIT = 3  # was 1
ALLOW_FILTER_CHANGES_IN_TWILIGHT = True  # was False
START_FILTER = FILTERS[0]
SUN_ALT_POLICY = [
    (-18.0, -15.0, ["g", "r", "i", "z", "y"]),
    (-15.0, -12.0, ["r", "i", "z", "y"]),
    (-12.0, 0.0, ["i", "r", "z", "y"]),
]

# -- Slew model --
SLEW_SMALL_DEG = 3.5
SLEW_SMALL_TIME_S = 4.0
SLEW_RATE_DEG_PER_S = 5.25
SLEW_SETTLE_S = 1.0

# -- Moon --
MIN_MOON_SEP_BY_FILTER = {
    "u": 80.0,
    "g": 50.0,
    "r": 35.0,
    "i": 30.0,
    "z": 25.0,
    "y": 20.0,
}
REQUIRE_SINGLE_TIME_FOR_ALL = True

# -- Time caps --
PER_SN_CAP_S = 120.0
MORNING_CAP_S = 'auto'
EVENING_CAP_S = 'auto'
TWILIGHT_STEP_MIN = 2
MAX_SN_PER_NIGHT = 100

# -- Priority strategy --
PRIORITY_STRATEGY = "hybrid"  # or "lc"
HYBRID_DETECTIONS = 2
HYBRID_EXPOSURE = 300.0
LC_DETECTIONS = 5
LC_EXPOSURE = 300.0

# -- Photometry / sky --
PIXEL_SCALE_ARCSEC = 0.2
ZPT1S = None
K_M = None
FWHM_EFF = None
READ_NOISE_E = 6.0
GAIN_E_PER_ADU = 1.0
ZPT_ERR_MAG = 0.01
DARK_SKY_MAG = None
TWILIGHT_DELTA_MAG = 2.5  # fallback if rubin_sim.skybrightness unavailable

# -- SIMLIB output --
SIMLIB_OUT = None  # e.g. "twilight.simlib"
SIMLIB_SURVEY = "LSST"
SIMLIB_FILTERS = "grizy"
SIMLIB_PIXSIZE = 0.2
SIMLIB_NPE_PIXEL_SATURATE = 1.0e6
SIMLIB_PHOTFLAG_SATURATE = 4096
SIMLIB_PSF_UNIT = "arcsec"

# -- Miscellaneous --
TYPICAL_DAYS_BY_TYPE = {
    "Ia": 70,
    "II-P": 100,
    "II-L": 70,
    "IIn": 120,
    "IIb": 70,
    "Ib": 60,
    "Ic": 60,
}
DEFAULT_TYPICAL_DAYS = 60
RA_COL = None
DEC_COL = None
DISC_COL = None
NAME_COL = None
TYPE_COL = None

# Assemble the configuration object with all parameters exposed
cfg = PlannerConfig(
    lat_deg=LAT_DEG,
    lon_deg=LON_DEG,
    height_m=HEIGHT_M,
    min_alt_deg=MIN_ALT_DEG,
    twilight_sun_alt_min_deg=TWILIGHT_SUN_ALT_MIN_DEG,
    twilight_sun_alt_max_deg=TWILIGHT_SUN_ALT_MAX_DEG,
    filters=FILTERS,
    carousel_capacity=CAROUSEL_CAPACITY,
    filter_change_s=FILTER_CHANGE_S,
    readout_s=READOUT_S,
    filter_change_time_s=FILTER_CHANGE_TIME_S,
    readout_time_s=READOUT_TIME_S,
    inter_exposure_min_s=INTER_EXPOSURE_MIN_S,
    exposure_by_filter=EXPOSURE_BY_FILTER,
    max_filters_per_visit=MAX_FILTERS_PER_VISIT,
    start_filter=START_FILTER,
    sun_alt_policy=SUN_ALT_POLICY,
    slew_small_deg=SLEW_SMALL_DEG,
    slew_small_time_s=SLEW_SMALL_TIME_S,
    slew_rate_deg_per_s=SLEW_RATE_DEG_PER_S,
    slew_settle_s=SLEW_SETTLE_S,
    min_moon_sep_by_filter=MIN_MOON_SEP_BY_FILTER,
    require_single_time_for_all_filters=REQUIRE_SINGLE_TIME_FOR_ALL,
    per_sn_cap_s=PER_SN_CAP_S,
    morning_cap_s=MORNING_CAP_S,
    evening_cap_s=EVENING_CAP_S,
    twilight_step_min=TWILIGHT_STEP_MIN,
    max_sn_per_night=MAX_SN_PER_NIGHT,
    cadence_enable=CADENCE_ENABLE,
    cadence_per_filter=CADENCE_PER_FILTER,
    cadence_days_target=CADENCE_DAYS_TARGET,
    cadence_jitter_days=CADENCE_JITTER_DAYS,
    cadence_days_tolerance=CADENCE_DAYS_TOLERANCE,
    cadence_bonus_sigma_days=CADENCE_BONUS_SIGMA_DAYS,
    cadence_bonus_weight=CADENCE_BONUS_WEIGHT,
    hybrid_detections=HYBRID_DETECTIONS,
    hybrid_exposure_s=HYBRID_EXPOSURE,
    lc_detections=LC_DETECTIONS,
    lc_exposure_s=LC_EXPOSURE,
    priority_strategy=PRIORITY_STRATEGY,
    pixel_scale_arcsec=PIXEL_SCALE_ARCSEC,
    zpt1s=ZPT1S,
    k_m=K_M,
    fwhm_eff=FWHM_EFF,
    read_noise_e=READ_NOISE_E,
    gain_e_per_adu=GAIN_E_PER_ADU,
    zpt_err_mag=ZPT_ERR_MAG,
    dark_sky_mag=DARK_SKY_MAG,
    twilight_delta_mag=TWILIGHT_DELTA_MAG,
    simlib_out=SIMLIB_OUT,
    simlib_survey=SIMLIB_SURVEY,
    simlib_filters=SIMLIB_FILTERS,
    simlib_pixsize=SIMLIB_PIXSIZE,
    simlib_npe_pixel_saturate=SIMLIB_NPE_PIXEL_SATURATE,
    simlib_photflag_saturate=SIMLIB_PHOTFLAG_SATURATE,
    simlib_psf_unit=SIMLIB_PSF_UNIT,
    typical_days_by_type=TYPICAL_DAYS_BY_TYPE,
    default_typical_days=DEFAULT_TYPICAL_DAYS,
    ra_col=RA_COL,
    dec_col=DEC_COL,
    disc_col=DISC_COL,
    name_col=NAME_COL,
    type_col=TYPE_COL,
    allow_filter_changes_in_twilight=ALLOW_FILTER_CHANGES_IN_TWILIGHT,
)

cfg

PlannerConfig(lat_deg=-30.2446, lon_deg=-70.7494, height_m=2663, min_alt_deg=20.0, twilight_sun_alt_min_deg=-15.0, twilight_sun_alt_max_deg=-5.0, filters=['g', 'r', 'i', 'z'], carousel_capacity=5, filter_change_s=120.0, readout_s=2.0, inter_exposure_min_s=15.0, filter_change_time_s=None, readout_time_s=None, exposure_by_filter={'g': 5.0, 'r': 5.0, 'i': 5.0, 'z': 5.0}, max_filters_per_visit=3, start_filter='g', sun_alt_policy=[(-18.0, -15.0, ['g', 'r', 'i', 'z', 'y']), (-15.0, -12.0, ['r', 'i', 'z', 'y']), (-12.0, 0.0, ['i', 'r', 'z', 'y'])], sun_alt_exposure_ladder=[], slew_small_deg=3.5, slew_small_time_s=4.0, slew_rate_deg_per_s=5.25, slew_settle_s=1.0, min_moon_sep_by_filter={'u': 80.0, 'g': 50.0, 'r': 35.0, 'i': 30.0, 'z': 25.0, 'y': 20.0}, require_single_time_for_all_filters=True, per_sn_cap_s=120.0, morning_cap_s='auto', evening_cap_s='auto', morning_twilight=None, evening_twilight=None, twilight_step_min=2, max_sn_per_night=100, hybrid_detections=2, hybrid_exposure_s=300.0, lc_d

## Run the planner

`plan_twilight_range_with_caps` is invoked below.  It reads the SN catalog from
`CSV_PATH`, writes output CSVs to `OUTDIR`, and prints progress information.
Two files are produced for each run:

- `lsst_twilight_plan_<run_label>_*.csv` — per-SN schedule
- `lsst_twilight_summary_<run_label>_*.csv` — per-night summary


In [10]:
# Execute the planner and return data frames
perSN_df, nights_df = plan_twilight_range_with_caps(
    csv_path=CSV_PATH,
    outdir=OUTDIR,
    start_date=START_DATE,
    end_date=END_DATE,
    cfg=cfg,
    run_label=RUN_LABEL,
    verbose=True,
)

print("Per-SN rows:", len(perSN_df), "  Nights summary rows:", len(nights_df))
perSN_df.head(10)

RA raw (numeric) range:  min=0.043167, max=359.855458
Dec raw (numeric) range: min=-87.141200, max=87.960305


Nights:   0%|          | 0/3 [00:00<?, ?night/s]

2024-01-01: eligible=155 visible=125 planned_total=93
  evening_twilight: 2024-01-02T00:11:00+00:00 → 2024-01-02T01:06:00+00:00 (local 19:28 → 20:23 UTC-04:43)
  morning_twilight: 2024-01-02T08:28:00+00:00 → 2024-01-02T09:23:00+00:00 (local 03:45 → 04:40 UTC-04:43)
2024-01-02: eligible=156 visible=129 planned_total=95
  evening_twilight: 2024-01-03T00:11:00+00:00 → 2024-01-03T01:06:00+00:00 (local 19:28 → 20:23 UTC-04:43)
  morning_twilight: 2024-01-03T08:28:00+00:00 → 2024-01-03T09:23:00+00:00 (local 03:45 → 04:40 UTC-04:43)
2024-01-03: eligible=155 visible=131 planned_total=39
  evening_twilight: 2024-01-04T00:11:00+00:00 → 2024-01-04T01:06:00+00:00 (local 19:28 → 20:23 UTC-04:43)
  morning_twilight: 2024-01-04T08:29:00+00:00 → 2024-01-04T09:24:00+00:00 (local 03:46 → 04:41 UTC-04:43)
Wrote:
  ../twilight_outputs/lsst_twilight_plan_<run_label>_2024-01-01_to_2024-01-03.csv
  ../twilight_outputs/lsst_twilight_summary_<run_label>_2024-01-01_to_2024-01-03.csv
  true-sequence: ../twilight

,date,twilight_window,SN,RA_deg,Dec_deg,best_twilight_time_utc,visit_start_utc,sn_end_utc,filter,t_exp_s,...,cadence_gate_passed,slew_s,cross_filter_change_s,filter_changes_s,readout_s,exposure_s,guard_s,inter_exposure_guard_enforced,total_time_s,elapsed_overhead_s
0,2024-01-01,evening,2023yxi,47.417777,-31.222880,2024-01-02T01:05:00+00:00,2024-01-02T00:11:00+00:00,2024-01-02T00:15:07+00:00,i,5.0,...,True,0.00,120.0,120.0,2.0,5.0,0.00,False,247.0,242.00
1,2024-01-01,evening,2023wde,53.804250,-32.423689,2024-01-02T01:05:00+00:00,2024-01-02T00:15:07+00:00,2024-01-02T00:15:27+00:00,i,5.0,...,True,5.39,0.0,0.0,2.0,5.0,9.61,True,20.0,5.39
2,2024-01-01,evening,2023abjs,54.716225,-34.363285,2024-01-02T01:05:00+00:00,2024-01-02T00:15:27+00:00,2024-01-02T00:15:47+00:00,i,5.0,...,True,5.00,0.0,0.0,2.0,5.0,10.00,True,20.0,5.00
3,2024-01-01,evening,2023xic,46.055083,-39.883689,2024-01-02T01:03:00+00:00,2024-01-02T00:15:47+00:00,2024-01-02T00:16:07+00:00,i,5.0,...,True,6.02,0.0,0.0,2.0,5.0,8.98,True,20.0,6.02
4,2024-01-01,evening,2023zzn,46.861701,-42.658672,2024-01-02T01:05:00+00:00,2024-01-02T00:16:07+00:00,2024-01-02T00:16:27+00:00,i,5.0,...,True,5.00,0.0,0.0,2.0,5.0,10.00,True,20.0,5.00
5,2024-01-01,evening,2023aael,39.128174,-45.880854,2024-01-02T00:35:00+00:00,2024-01-02T00:16:27+00:00,2024-01-02T00:16:47+00:00,i,5.0,...,True,5.55,0.0,0.0,2.0,5.0,9.45,True,20.0,5.55
6,2024-01-01,evening,2023yqc,38.192499,-38.310448,2024-01-02T00:33:00+00:00,2024-01-02T00:16:47+00:00,2024-01-02T00:17:07+00:00,i,5.0,...,True,5.78,0.0,0.0,2.0,5.0,9.22,True,20.0,5.78
7,2024-01-01,evening,2023xjl,28.083711,-36.662630,2024-01-02T00:11:00+00:00,2024-01-02T00:17:07+00:00,2024-01-02T00:17:27+00:00,i,5.0,...,True,5.89,0.0,0.0,2.0,5.0,9.11,True,20.0,5.89
8,2024-01-01,evening,2023xbc,18.756059,-33.018606,2024-01-02T00:11:00+00:00,2024-01-02T00:17:27+00:00,2024-01-02T00:17:47+00:00,i,5.0,...,True,5.95,0.0,0.0,2.0,5.0,9.05,True,20.0,5.95
9,2024-01-01,evening,2023zrc,17.044547,-36.059907,2024-01-02T00:11:00+00:00,2024-01-02T00:17:47+00:00,2024-01-02T00:18:07+00:00,i,5.0,...,True,5.00,0.0,0.0,2.0,5.0,10.00,True,20.0,5.00


In [11]:
# Show the nightly summary
nights_df

,date,twilight_window,n_candidates,n_planned,sum_time_s,window_cap_s,swap_count,internal_filter_changes,filter_change_s_total,inter_exposure_guard_s,...,policy_filters_mid_csv,window_utilization,cap_utilization,cap_source,median_sky_mag_arcsec2,median_alt_deg,cad_median_abs_err_by_filter_csv,cad_within_pct_by_filter_csv,cad_median_abs_err_all_d,cad_within_pct_all
0,2024-01-01,evening,78,78,1787.0,3300,1,0,240.0,695.1,...,"i,z",0.5415,0.5415,window_duration,18.992206,60.430,,,NaN,NaN
1,2024-01-01,morning,22,15,533.6,3300,1,0,240.0,112.3,...,"i,z",0.1617,0.1617,window_duration,19.831240,73.280,,,NaN,NaN
2,2024-01-02,evening,77,77,2006.3,3300,2,0,480.0,679.2,...,"i,z",0.6080,0.6080,window_duration,18.625368,59.860,,,NaN,NaN
3,2024-01-02,morning,23,18,817.9,3300,2,0,480.0,134.8,...,"i,z",0.2479,0.2479,window_duration,19.230536,62.190,,,NaN,NaN
4,2024-01-03,evening,62,12,710.2,3300,2,0,480.0,88.4,...,"i,z",0.2152,0.2152,window_duration,18.792881,24.985,,,NaN,NaN
5,2024-01-03,morning,38,27,998.6,3300,2,0,480.0,216.2,...,"i,z",0.3026,0.3026,window_duration,19.626695,32.440,,,NaN,NaN



## Inspect outputs on disk


In [12]:
import glob
import os

files = sorted(glob.glob(os.path.join(OUTDIR, "lsst_twilight_*.csv")))
files

['../twilight_outputs/lsst_twilight_plan_2024-01-01_to_2024-01-03.csv',
 '../twilight_outputs/lsst_twilight_sequence_true_2024-01-01_to_2024-01-03.csv',
 '../twilight_outputs/lsst_twilight_summary_2024-01-01_to_2024-01-03.csv']